In [34]:
import pandas as pd
from geopy.distance import geodesic
import numpy as np
import matplotlib.pyplot as plt
import pickle
from datetime import datetime, timedelta

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.cluster import KMeans
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample
from sklearn.svm import SVC


clean data. make rolling window

In [53]:
full_derates_seasons = pd.read_csv('data/full_derates_seasons.csv', low_memory=False)

In [54]:
full_derates_seasons = full_derates_seasons.drop(columns = 'Unnamed: 0')

In [55]:
full_derates_seasons = full_derates_seasons.set_index('EventTimeStamp').sort_index(ascending = False)

In [56]:
full_derates_seasons = full_derates_seasons[['spn', 'AcceleratorPedal','EngineRpm','EquipmentID', 'EngineOilTemperature',
                                             'EngineLoad', 'EngineOilPressure','fmi']]

# Convert the columns to numeric data types
full_derates_seasons = full_derates_seasons.apply(pd.to_numeric, errors='coerce')

# Fill NaN values with 0 and convert to integers
full_derates_seasons = full_derates_seasons.fillna(0).astype(int)

In [60]:
full_derates_seasons_dum = pd.get_dummies(data=full_derates_seasons.loc[:, ['spn',
                                                                
                                                                            #'AcceleratorPedal',
                                                                            #'EngineRpm',
                                                                            'EquipmentID', 
                                                                            #'EngineOilTemperature',
                                                                            #'EngineLoad', 
                                                                            #'EngineOilPressure',
                                                                            'fmi'
                                                                           ]], columns=['spn'])                                        
                                                            

In [61]:
full_derates_seasons_dum = full_derates_seasons_dum.replace({'unknown': '0'})

In [62]:
full_derates_rolling = (full_derates_seasons_dum
                  .groupby(by = 'EquipmentID')
                        .rolling(window = 1)
                        .sum()
                 )

model 1: RandomForestClassifier

In [63]:
# Separate majority and minority classes
df_majority = full_derates_rolling[full_derates_rolling.spn_5246==0]
df_minority = full_derates_rolling[full_derates_rolling.spn_5246==1]
 
# Downsample majority class
df_majority_downsampled = resample(df_majority, 
                                 replace=False,    # sample without replacement
                                 n_samples=2000,     # to match minority class
                                 random_state=200) # reproducible results
 
# Combine minority class with downsampled majority class
df_downsampled = pd.concat([df_majority_downsampled, df_minority])
 
# Display new class counts
df_downsampled.spn_5246.value_counts()

spn_5246
0.0    2000
1.0     930
Name: count, dtype: int64

In [64]:
X = df_downsampled.drop(columns='spn_5246')

y = df_downsampled['spn_5246']

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=.25, random_state = 321)

In [65]:
scaler = StandardScaler()
imputer = SimpleImputer()
rfc = RandomForestClassifier  # LogisticRegression

In [66]:
steps = [("imputer", imputer), 
         ("scaler", scaler),
         ("rfc", RandomForestClassifier(n_estimators=100, max_depth = 100
                    )
         )
         
        ]


pipeline = Pipeline(steps = steps)

In [67]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('imputer', SimpleImputer()), ('scaler', StandardScaler()),
                ('rfc', RandomForestClassifier(max_depth=100))])

In [68]:
print(classification_report(y_test, pipeline.predict(X_test)))

              precision    recall  f1-score   support

         0.0       1.00      0.99      0.99       500
         1.0       0.98      1.00      0.99       233

    accuracy                           0.99       733
   macro avg       0.99      0.99      0.99       733
weighted avg       0.99      0.99      0.99       733



In [69]:
print(confusion_matrix(y_test, pipeline.predict(X_test)))

[[495   5]
 [  0 233]]


In [70]:
X2 = full_derates_rolling.drop(columns='spn_5246')

y2 = full_derates_rolling['spn_5246']

X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=.25, random_state = 321)

In [71]:
scaler = StandardScaler()
imputer = SimpleImputer()
rfc = RandomForestClassifier 

In [72]:
steps = [("imputer", imputer), 
         ("scaler", scaler),
         ("rfc", RandomForestClassifier(n_estimators=100, max_depth = 200
                    )
         )
         
        ]


pipeline = Pipeline(steps = steps)

In [73]:
pipeline.fit(X2_train, y2_train)

Pipeline(steps=[('imputer', SimpleImputer()), ('scaler', StandardScaler()),
                ('rfc', RandomForestClassifier(max_depth=200))])

In [74]:
print(classification_report(y2_test, pipeline.predict(X2_test)))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00    264123
         1.0       0.98      1.00      0.99       243

    accuracy                           1.00    264366
   macro avg       0.99      1.00      0.99    264366
weighted avg       1.00      1.00      1.00    264366



In [75]:
print(confusion_matrix(y2_test, pipeline.predict(X2_test)))


[[264118      5]
 [     0    243]]


In [49]:
#scores = cross_val_score(rfc, X2, y2, cv=5, scoring='f1')

In [ ]:
# true neg = top left 
# false pos = top right
# false neg = bottom left
# true pos = bottom right 